In [2]:
from datasets import load_dataset, Audio, load_from_disk
from transformers import EncodecModel, AutoProcessor




# load the dataset
dataset_new = load_dataset("dthomas84/rule1_embeddings")
# load the model + processor (for pre-processing the audio)
model = EncodecModel.from_pretrained("facebook/encodec_32khz")
processor = AutoProcessor.from_pretrained("facebook/encodec_32khz")


def encode_clips(dataset=dataset, model = model, processor = processor):
    dataset_cast = dataset.cast_column("audio", Audio(sampling_rate=processor.sampling_rate))
    audio_outputs = [()]
    i = 0
    while i < 4:
    #for i in range(len(dataset_cast)):
        audio_sample = dataset_cast["audio"][i]["array"]
        inputs = processor(raw_audio=audio_sample, sampling_rate=processor.sampling_rate, return_tensors="pt")
        encoder_outputs = model.encode(inputs["input_values"], inputs["padding_mask"])
        #audio_values = model.decode(encoder_outputs.audio_codes, encoder_outputs.audio_scales, inputs["padding_mask"])[0]
        #dataset_cast["audio"][i]["array"] = audio_values
        # Assuming `audio_codes` is your tensor
        audio_codes_flat = encoder_outputs.audio_codes.flatten().cpu().numpy()
        audio_outputs.append([(audio_codes_flat, encoder_outputs.audio_scales)])
        i += 1

    return audio_outputs




dataset_cast = dataset_new.cast_column("audio", Audio(sampling_rate=processor.sampling_rate))




Resolving data files:   0%|          | 0/101 [00:00<?, ?it/s]

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


NameError: name 'dataset' is not defined

In [8]:
# Load in the .jsonl file
dataset = load_from_disk("./dataset/")

In [3]:
# Load in the .jsonl file
with open ("./data/metadata.jsonl", "r") as f:
    dataset = f.readlines()

In [4]:
import json
dataset = [json.loads(line) for line in dataset]

In [24]:
len(dataset)

283

In [26]:
import librosa
i=0
# Convert the "file_name" for each sample to a np.array
for i in range(len(dataset)):
    dataset[i]["audio"] = librosa.load(dataset[i]["file_name"], sr=32000)[0]
    i += 1

In [27]:
for data in dataset:
    print(len(data["audio"]))

480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000
480000

In [8]:
for output in audio_outputs:
    print(output)


()
[(array([ 166,  166,  472, ..., 2010, 1013, 1903], dtype=int64), [None])]
[(array([ 461,   68, 1358, ...,  446, 1612,   56], dtype=int64), [None])]
[(array([ 673, 1047,  475, ...,  540,    1,   57], dtype=int64), [None])]
[(array([ 974,   16, 1047, ...,  886,  193, 1887], dtype=int64), [None])]


In [23]:
from datasets import load_dataset

dataset = load_dataset("./dataset/data/")

Resolving data files:   0%|          | 0/284 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [5]:
# Cast the "audio" column to Audio
dataset_cast = dataset.cast_column("audio", Audio(sampling_rate=processor.sampling_rate))

AttributeError: 'list' object has no attribute 'cast_column'

In [211]:
encoder_outputs.audio_scales

[None]

In [28]:
audio_outputs = []
i = 10
for i in range(10, len(dataset)):
    audio_sample = dataset[i]["audio"]
    inputs = processor(raw_audio=audio_sample, sampling_rate=processor.sampling_rate, return_tensors="pt")
    encoder_outputs = model.encode(inputs["input_values"], inputs["padding_mask"])
    #audio_values = model.decode(encoder_outputs.audio_codes, encoder_outputs.audio_scales, inputs["padding_mask"])[0]
    #dataset_cast["audio"][i]["array"] = audio_values
    # Assuming `audio_codes` is your tensor
    audio_codes_flat = encoder_outputs.audio_codes.flatten().cpu().numpy()
    dataset[i]["audio"] = audio_codes_flat
    audio_outputs.append(audio_codes_flat)
    i += 1


3000
3000
3000
3000
3000
3000
3000
3000
3000
3000


In [152]:
import torch
# Convert the first value of the "audio_outputs" list to a tensor
audio_tensor = torch.tensor(audio_outputs[0])
# Reshape the tensor to the correct shape
#audio = model.decode(audio_tensor, audio_scales = [0])
audio_tensor.reshape(1, 0).shape   

C:\Users\sjufa\AppData\Local\Temp\ipykernel_16392\2415050758.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  audio_tensor = torch.tensor(audio_outputs[0])


RuntimeError: shape '[1, 0]' is invalid for input of size 1

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [56]:
for i, data in enumerate(dataset[:len(audio_outputs)]):
    # Set the "audio" column to the encoded audio
    data["audio"] = audio_outputs[i]

[ 166  166  472 ... 2010 1013 1903]
[ 461   68 1358 ...  446 1612   56]


In [30]:
import os
import pinecone
from dotenv import load_dotenv
load_dotenv()

pinecone_key = os.getenv("PINECONE_KEY")
pinecone_environment = os.getenv("PINECONE_ENV")

pinecone.init(api_key=pinecone_key, environment=pinecone_environment)


In [31]:
index = pinecone.Index(index_name="musicgen")



In [171]:
# Retrieve the first 10 results
query_results = index.query(audio_codes_flat.tolist(), top_k=1, include_values=True)


In [270]:
codes = query_results["matches"][0]["values"] 

In [271]:
import numpy as np
# Convert the codes to a numpy array
codes_np = np.array(codes)
# Reshape the array to the correct shape
original_shape = (1, 1, 4, 750)
retrieved_audio_codes = np.reshape(codes_np, original_shape)


In [272]:
# Convert the codes tensor to int 
codes_int = retrieved_audio_codes_tensor.int()

In [273]:
  # Replace `device` with actual device like "cuda"
# Step 4: Decode
decoded_audio = model.decode(
    audio_codes=codes_int,
    audio_scales=[None]  # Assuming this tensor is available from your encoding step
      # Assuming this tensor is available from your encoding step
)  # Assuming you are interested in `audio_values`


In [274]:
decoded_audio   

EncodecDecoderOutput(audio_values=tensor([[[ 0.0378,  0.0327,  0.0581,  ..., -0.0768, -0.0797, -0.0438]]],
       grad_fn=<ConvolutionBackward0>))

In [219]:
# Load an audio clip and split it into 15 sec chunks
import librosa
import numpy as np
audio, sr = librosa.load("./Avicci - Wake Me Up.mp3", sr=32000)
audio_chunks = np.array_split(audio, len(audio) // (15 * sr))

In [220]:
# If there are any chunks that are less than 15 seconds pad them with zeros
if len(audio_chunks[-1]) < 15 * sr:
    audio_chunks[-1] = np.pad(audio_chunks[-1], (0, 15 * sr - len(audio_chunks[-1])), "constant")

# Encode the audio chunks
encoded_audio_chunks = []
for chunk in audio_chunks:
    inputs = processor(raw_audio=chunk, sampling_rate=processor.sampling_rate, return_tensors="pt")
    encoder_outputs = model.encode(inputs["input_values"], inputs["padding_mask"])
    encoded_audio_chunks.append(encoder_outputs.audio_codes.flatten().cpu().numpy())

In [282]:
encoded_audio_chunks[1] = encoded_audio_chunks[1][:3000]

In [283]:
len(encoded_audio_chunks[1])

3000

In [284]:
results = index.query(encoded_audio_chunks[1].tolist(), top_k=1, include_values=True)


In [246]:
# Query the vector index with the first chunk
query_results = index.query(encoded_audio_chunks[0].tolist(), top_k=2, include_values=True)

In [258]:
codes = results["matches"][0]["values"] 


In [32]:
from pydantic import BaseModel, Field
from typing import List, Optional
import json

# Create a class for the lyrics object
class SongClip(BaseModel):
    """ SongClip object. """
    lyrics: Optional[str] = Field(description="Lyrics of the song")
    title: str = Field(description="Title of the song")
    artist: str = Field(description="Artist of the song")
    album: Optional[str] = Field(description="Album of the song")
    source: str = Field(description="Source of the song")
    music: Optional[List[float]] = Field(
                description="Music of the song")
    description: Optional[str] = Field(
                description="Description of the song")

# Create a function to embed the lyrics and upsert
# them into our Pinecone index
def upsert_music(songs_list: Optional[List[SongClip]] = None):
    """ Upsert the lyrics into the Pinecone index. """
    #    {"album": "Under The Table And Dreaming", "artist": "Dave Matthews Band", "description": "A fan favorite at shows, the song changes the progression several times through out it.", "file_name": "./data/Warehouse_26.wav", "is_padded": false, "lyrics": "10 ContributorsWarehouse Lyrics[Intro]\nSaid I'm leaving\nThis warehouse frightens me\nGot me tied up in knots, can't rest for one moment\nSaid I'm going\nNow I'm slipping slow away\nHoping to find something better than I've got inside of here\nAnd the warehouse slips away\n\n[Instrumental]\n\n[Verse]\nHey reckless mind\nDon't throw away your playful beginnings\nYou and I, will fumble around in the touching\nAnd be sure to leave all the lights on\nSo we can see the black cat changing colors\nAnd we can walk under ladders\nAnd swim as the tide turns you around and around  , love\n\nHey we have found\nBecoming one in a million\nSlip into the crowd\nThis question I found in a gap in the sidewalk\nKeep all your sights on\nHey the black cat changing colors\nAnd you can walk under ladders\nAnd swim as the tide choose to turn you away\nSaid, Life goes on\nEnd of tunnel, TV sets\nSpot in the middle\nStatic fade, statistical bit\nAnd soon I'll fade away, I'll fade away\nOh, but this I admit tastes so good\nHard to believe an end to it\nSmell touch feel\nHow could this rhythm ever quit\nBags packed on a plane\nHopefully to heaven, yeah\nSee Dave Matthews Band LiveGet tickets as low as $71You might also like[Instrumental]\n\n[Verse]\nShut up I'm thinking\nI had a clue, now it's gone forever\nSitting over these bones\nYou can read in whatever you're needing to\nKeep all your sights on\nYeah man the black cat changing colors\nWhen it's not the colors that matter\nBut that they'll all fade away\nYeah, yeah, yeah\nAye, Life goes on\nEnd of tunnel, TV sets\nSpot in the middle\nStatic fade, statistical bit\nAnd soon I'll fade away, I'll fade away\nOh, and this I admit seems so good\nHard to believe an end to it\nWarehouse is bare\nNothing at all inside of it\nThe walls and halls have disappeared\nThey've disappeared where?\n\nOoh, I'd love to stay, yeah\nOoh, I'd love to stay, yeah\nOoh, I'd love to stay, yeah\nOoh, I'd love to stay, yeah\nIn a corner and wondering\nIf a change can be better than this\nOh, and then worry\nMaybe things won't be better than they have been here in the warehouse\nAt the warehouse\nAt the warehouse\nOh, how I'd love to stay here at the warehouse\nEvery man and woman, get alive\n\n[Instrumental]\n\n[Verse]\nThat's our blood down there\nSeems poured from the hands of angels\nBut trickled into the ground\nLeave the warehouse bare and empty\nAnd my heart's numbered beats\nStill echo in this empty room\nAnd the fear wells in me\nBut nothing seems good enough to them\nSo I am going away\nI'm going away\nAnd I'm not giving in5Embed", "segment_num": 26, "source": "https://genius.com/Dave-matthews-band-warehouse-lyrics", "title": "Warehouse"}
    # Load in the .jsonl file 
    #with open("./dataset/metadata.jsonl", "r") as f:
    #    for line in f:
    #        song = json.loads(line)
    #        songs_list.append(song)
    i=0    
    # Loop through the songs
    for i in range(10, len(dataset)):
        # Define the vectors
        vectors = [
            {
            "metadata": {
                "title": dataset[i]["title"],
                "artist": dataset[i]["artist"],
                "album": dataset[i]["album"],
                "source": dataset[i]["source"],
                "description": dataset[i]["description"]
                },
            "values": dataset[i]["audio"].astype(float).tolist(),
            "id": f"{dataset[i]['title']}_{i}",
            }
        ]

        # Upsert the lyrics
        index.upsert(vectors=vectors)
        i += 1

    # Return a success message and the index name statistics
    return "Success!"

upsert_music()  

'Success!'

In [251]:
len(codes)

3000

In [38]:
len()



dict_keys(['album', 'artist', 'description', 'file_name', 'is_padded', 'lyrics', 'segment_num', 'source', 'title'])
dict_keys(['album', 'artist', 'description', 'file_name', 'is_padded', 'lyrics', 'segment_num', 'source', 'title'])
dict_keys(['album', 'artist', 'description', 'file_name', 'is_padded', 'lyrics', 'segment_num', 'source', 'title'])
dict_keys(['album', 'artist', 'description', 'file_name', 'is_padded', 'lyrics', 'segment_num', 'source', 'title'])


AttributeError: 'str' object has no attribute 'keys'

In [30]:

# pre-process the inputs
inputs = processor(raw_audio=audio_sample, sampling_rate=processor.sampling_rate, return_tensors="pt")

# explicitly encode then decode the audio inputs
encoder_outputs = model.encode(inputs["input_values"], inputs["padding_mask"])
audio_values = model.decode(encoder_outputs.audio_codes, encoder_outputs.audio_scales, inputs["padding_mask"])[0]

# or the equivalent with a forward pass
audio_values = model(inputs["input_values"], inputs["padding_mask"]).audio_values

# you can also extract the discrete codebook representation for LM tasks
# output: concatenated tensor of all the representations
audio_codes = model(inputs["input_values"], inputs["padding_mask"]).audio_codes

AttributeError: 'MusicgenProcessor' object has no attribute 'sampling_rate'

In [14]:
decoded = model.decode(audio_codes, audio_scales = encoder_outputs.audio_scales)

In [263]:
# Convert the tensor to a numpy array
decoded = decoded_audio[0].cpu().detach().numpy()

In [264]:
from IPython.display import Audio
# Flatten the numpy array to 2d
decoded = decoded.reshape(-1)
Audio(decoded, rate=processor.sampling_rate)

In [48]:
outputs_list1

# Return to a 2d array
outputs_list1 = outputs_list1.reshape(1, -1)

In [49]:
outputs_list1.shape

(1, 2250)

In [22]:
# Convert the numpy array back to a tensor
import torch

outputs_list_flat_tensor = torch.tensor(outputs_list_flat)
outputs_list_flat_tensor.shape

C:\Users\sjufa\AppData\Local\Temp\ipykernel_22932\2921140256.py:4: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\b\abs_abjetg6_iu\croot\pytorch_1686932924616\work\torch\csrc\utils\tensor_new.cpp:248.)
  outputs_list_flat_tensor = torch.tensor(outputs_list_flat)


torch.Size([18, 2250])